In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from sparkmeasure import StageMetrics
spark=SparkSession \
        .builder \
        .master("spark://ubuntu:7077")\
        .appName("Top 10 movies for every year")\
        .getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [2]:
stagemetrics = StageMetrics(spark)

In [3]:
stagemetrics.begin()
Ratings_Dataframe=(spark.read.format("csv")
            .option("header","True")
            .option("inferSchema","True")
            .load("/home/meli/Documents/Kaggle/rating.csv")
      )
Movies_Dataframe=(spark.read.format("csv")
            .option("header","True")
            .option("inferSchema","True")
            .load("/home/meli/Documents/Kaggle/movie.csv")
      )
Ratings_Dataframe=Ratings_Dataframe.withColumn("timestamp",date_format("timestamp","yyy"))
tdf=(Ratings_Dataframe.groupBy(["timestamp","movieId"]).avg("rating")
     .join(Movies_Dataframe,Movies_Dataframe.movieId==Ratings_Dataframe.movieId)
     .drop(Movies_Dataframe.movieId)
     .drop(Movies_Dataframe.genres) 
     .sort("timestamp")
    )
windowDept = Window.partitionBy("timestamp").orderBy(col("avg(rating)").desc())
FullQuery=(tdf.withColumn("row",row_number().over(windowDept))
           .filter(col("row") <= 10)
           .drop("row")
          )
FullQuery.where("timestamp==2005").sort("title").show(truncate=0)
stagemetrics.end()

+---------+-------+-----------+----------------------------------------------------------------------------+
|timestamp|movieId|avg(rating)|title                                                                       |
+---------+-------+-----------+----------------------------------------------------------------------------+
|2005     |33124  |5.0        |Before the Fall (NaPolA - Elite für den Führer) (2004)                      |
|2005     |2538   |5.0        |Dancemaker (1998)                                                           |
|2005     |6236   |5.0        |Fear Strikes Out (1957)                                                     |
|2005     |787    |5.0        |Gate of Heavenly Peace, The (1995)                                          |
|2005     |771    |5.0        |Life Is Rosy (a.k.a. Life Is Beautiful) (Vie est belle, La) (1987)          |
|2005     |6309   |5.0        |Married to It (1991)                                                        |
|2005     |793    |

In [4]:
stagemetrics.print_report()


Scheduling mode = FIFO
Spark Context default degree of parallelism = 4
Aggregated Spark stage metrics:
numStages => 9
numTasks => 418
elapsedTime => 74344 (1.2 min)
stageDuration => 71553 (1.2 min)
executorRunTime => 209588 (3.5 min)
executorCpuTime => 87844 (1.5 min)
executorDeserializeTime => 17566 (18 s)
executorDeserializeCpuTime => 4084 (4 s)
resultSerializationTime => 226 (0.2 s)
jvmGCTime => 2755 (3 s)
shuffleFetchWaitTime => 26 (26 ms)
shuffleWriteTime => 1277 (1 s)
resultSize => 1270208 (1240.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 2285992000
recordsRead => 40055086
bytesRead => 1384480482 (1320.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 92597
shuffleTotalBlocksFetched => 2600
shuffleLocalBlocksFetched => 2600
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 1827333 (1784.0 KB)
shuffleLocalBytesRead => 1827333 (1784.0 KB)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesRead